In [ ]:
import py4dgeo
import numpy as np
from datetime import datetime 
import os
from pathlib import Path
import open3d as o3d

In [ ]:

path = "res/point_clouds"
pattern = "dense_*.ply"

first_n = 50
pc_list = sorted(Path(path).glob(f"{pattern}"))
pc_list = pc_list[:first_n]

# read the timestamps from file names
timestamps = []
for f in pc_list:

    # get the timestamp from the file name
    timestamp_str = f.stem.replace("dense_", "")

    # convert string to datetime object
    timestamp = datetime.strptime(timestamp_str, '%Y_%m_%d')
    timestamps.append(timestamp)


In [ ]:
analysis = py4dgeo.SpatiotemporalAnalysis('test.zip', force=True)

In [ ]:
pcd_ref  = o3d.io.read_point_cloud(str(pc_list[0])).random_down_sample(0.3)
pcd_ref = np.asarray(pcd_ref.points)

reference_epoch = py4dgeo.epoch.as_epoch(pcd_ref)
reference_epoch.timestamp = timestamps[0]

In [ ]:
analysis.reference_epoch  = reference_epoch

In [ ]:
# specify corepoints, here all points of the reference epoch
analysis.corepoints = reference_epoch.cloud[::100]

# specify M3C2 parameters
analysis.m3c2 = py4dgeo.M3C2(cyl_radii=(0.5,), normal_radii=(0.3,), max_distance=2.0, registration_error = 0.15)

In [ ]:
# create a list to collect epoch objects
epochs = []
for e, pc_file in enumerate(pc_list[1:]):
    pcd  = o3d.io.read_point_cloud(str(pc_file)).random_down_sample(0.3)
    pcd = np.asarray(pcd.points)    
    epoch = py4dgeo.epoch.as_epoch(pcd) 
    epoch.timestamp = timestamps[e]
    epochs.append(epoch)

# add epoch objects to the spatiotemporal analysis object
analysis.add_epochs(*epochs)

In [ ]:
analysis = py4dgeo.SpatiotemporalAnalysis('test.zip')

In [ ]:
# print the spatiotemporal analysis data for 3 corepoints and 5 epochs, respectively
print(f"Space-time distance array:\n{analysis.distances[:3,:5]}")
print(f"Uncertainties of M3C2 distance calculation:\n{analysis.uncertainties['lodetection'][:3, :5]}")
print(f"Timestamp deltas of analysis:\n{analysis.timedeltas[:5]}")

In [ ]:
# import plotting module
import matplotlib.pyplot as plt

# allow interactive rotation in notebook
%matplotlib inline

# create the figure
fig=plt.figure(figsize=(12,5))
ax1=fig.add_subplot(1,2,1,projection='3d',computed_zorder=False)
ax2=fig.add_subplot(1,2,2)

# get the corepoints
corepoints = analysis.corepoints.cloud

# get change values of last epoch for all corepoints
distances = analysis.distances
distances_epoch = [d[20] for d in distances]

# get the time series of changes at a specific core point locations
cp_idx_sel = 10000
coord_sel = analysis.corepoints.cloud[cp_idx_sel]
timeseries_sel = distances[cp_idx_sel]

# get the list of timestamps from the reference epoch timestamp and timedeltas
timestamps = [t + analysis.reference_epoch.timestamp for t in analysis.timedeltas]

# plot the scene
d = ax1.scatter(corepoints[:,0], corepoints[:,1], corepoints[:,2], c=distances_epoch[:], cmap='seismic_r', vmin=-1.5, vmax=1.5, s=1, zorder=1) 
plt.colorbar(d, format=('%.2f'), label='Distance [m]', ax=ax1, shrink=.5, pad=.15)

# add the location of the selected coordinate
ax1.scatter(coord_sel[0], coord_sel[1], coord_sel[2], c='black', s=3, zorder=2, label='Selected location')
ax1.legend()

ax1.set_xlabel('X [m]')
ax1.set_ylabel('Y [m]')
ax1.set_zlabel('Z [m]')
ax1.set_aspect('equal')
ax1.view_init(elev=30., azim=150.)
ax1.set_title('Changes at %s' % (analysis.reference_epoch.timestamp+analysis.timedeltas[20]))

# plot the time series
ax2.plot(timestamps, timeseries_sel, color='blue')
ax2.set_xlabel('Date')
ax2.set_ylabel('Distance [m]')
ax2.grid()
# ax2.set_ylim(-0.2,1.0)
ax2.set_title('Time series at selected location')

plt.tight_layout()
plt.show()